In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import multiprocessing as mp
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
import calendar
from scipy.sparse import csr_matrix,hstack
#import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from tqdm import tqdm
import pickle

#Reading up the dataframes
train=pd.read_csv('final_dataframe.csv')
test=pd.read_csv('final_dataframe_test.csv')
final_test=pd.read_csv('final_future_data.csv')

lbl=LabelEncoder()
train['item_id']=lbl.fit_transform(train['item_id'])
test['item_id']=lbl.transform(test['item_id'])
final_test['item_id']=lbl.transform(final_test['item_id'])
pickle.dump(lbl,open('label_encoder_item_id.sav','wb'))

lbl=LabelEncoder()
train['dept_id']=lbl.fit_transform(train['dept_id'])
test['dept_id']=lbl.transform(test['dept_id'])
final_test['dept_id']=lbl.transform(final_test['dept_id'])
pickle.dump(lbl,open('label_encoder_dept_id.sav','wb'))

lbl=LabelEncoder()
train['cat_id']=lbl.fit_transform(train['cat_id'])
test['cat_id']=lbl.transform(test['cat_id'])
final_test['cat_id']=lbl.transform(final_test['cat_id'])
pickle.dump(lbl,open('label_encoder_cat_id.sav','wb'))

lbl=LabelEncoder()
train['store_id']=lbl.fit_transform(train['store_id'])
test['store_id']=lbl.transform(test['store_id'])
final_test['store_id']=lbl.transform(final_test['store_id'])
pickle.dump(lbl,open('label_encoder_store_id.sav','wb'))

lbl=LabelEncoder()
train['state_id']=lbl.fit_transform(train['state_id'])
test['state_id']=lbl.transform(test['state_id'])
final_test['state_id']=lbl.transform(final_test['state_id'])
pickle.dump(lbl,open('label_encoder_state_id.sav','wb'))

lbl=LabelEncoder()
train['event_name_1']=lbl.fit_transform(train['event_name_1'])
test['event_name_1']=lbl.transform(test['event_name_1'])
final_test['event_name_1']=lbl.transform(final_test['event_name_1'])
pickle.dump(lbl,open('label_encoder_event_name_1.sav','wb'))

lbl=LabelEncoder()
train['event_name_2']=lbl.fit_transform(train['event_name_2'])
test['event_name_2']=lbl.transform(test['event_name_2'])
final_test['event_name_2']=lbl.transform(final_test['event_name_2'])
pickle.dump(lbl,open('label_encoder_event_name_2.sav','wb'))

lbl=LabelEncoder()
train['event_type_1']=lbl.fit_transform(train['event_type_1'])
test['event_type_1']=lbl.transform(test['event_type_1'])
final_test['event_type_1']=lbl.transform(final_test['event_type_1'])
pickle.dump(lbl,open('label_encoder_event_type_1.sav','wb'))

lbl=LabelEncoder()
train['event_type_2']=lbl.fit_transform(train['event_type_2'])
test['event_type_2']=lbl.transform(test['event_type_2'])
final_test['event_type_2']=lbl.transform(final_test['event_type_2'])
pickle.dump(lbl,open('label_encoder_event_type_2.sav','wb'))

lbl=LabelEncoder()
train['year']=lbl.fit_transform(train['year'])
test['year']=lbl.transform(test['year'])
final_test['year']=lbl.transform(final_test['year'])
pickle.dump(lbl,open('label_encoder_year.sav','wb'))

%%time
# Converting snap_CA,snap_WI,snap_TX into one feature named snap
train.loc[train['state_id'] == 'CA', 'snap'] = train.loc[train['state_id'] == 'CA']['snap_CA']
train.loc[train['state_id'] == 'TX', 'snap'] = train.loc[train['state_id'] == 'TX']['snap_TX']
train.loc[train['state_id'] == 'WI', 'snap'] = train.loc[train['state_id'] == 'WI']['snap_WI']
train.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)


test.loc[test['state_id'] == 'CA', 'snap'] = test.loc[test['state_id'] == 'CA']['snap_CA']
test.loc[test['state_id'] == 'TX', 'snap'] = test.loc[test['state_id'] == 'TX']['snap_TX']
test.loc[test['state_id'] == 'WI', 'snap'] = test.loc[test['state_id'] == 'WI']['snap_WI']
test.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

final_test.loc[final_test['state_id'] == 'CA', 'snap'] = final_test.loc[final_test['state_id'] == 'CA']['snap_CA']
final_test.loc[final_test['state_id'] == 'TX', 'snap'] = final_test.loc[final_test['state_id'] == 'TX']['snap_TX']
final_test.loc[final_test['state_id'] == 'WI', 'snap'] = final_test.loc[final_test['state_id'] == 'WI']['snap_WI']
final_test.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time
# Weekday as wday are similar features so we remove it
#I will make no use of  wm_yr_wk feature
train.drop('weekday',axis=1,inplace=True)
train.drop('wm_yr_wk',axis=1,inplace=True)
 
test.drop('weekday',axis=1,inplace=True)
test.drop('wm_yr_wk',axis=1,inplace=True)

final_test.drop('weekday',axis=1,inplace=True)
final_test.drop('wm_yr_wk',axis=1,inplace=True)

Wall time: 6.82 s


## Date related features

### Week Number

In [ ]:
#Reference https://stackoverflow.com/questions/2600775/how-to-get-week-number-in-python/32638267#:~:text=()%5B1%5D%2024-,datetime.,for%20the%20given%20date%20instance.&text=You%20can%20get%20the%20week%20number%20directly%20from%20datetime%20as%20string.
def get_week_number(x):
    """This Function is used to get weeknumber of particular date"""
    date=calendar.datetime.date.fromisoformat(x)
    return date.isocalendar()[1]

In [ ]:
train['week_number']=train['date'].apply(lambda x:get_week_number(x))
test['week_number']=test['date'].apply(lambda x:get_week_number(x))
final_test['week_number']=final_test['date'].apply(lambda x:get_week_number(x))

### Season

In [ ]:
#https://www.universaltraveller.com.au/destinations/los-angeles/weather
def get_season(x):
    """This function is used to get season in US according to various months"""
    if x in [12,1,2]:
        return 0      #"Winter"
    elif x in [3,4,5]:
        return 1   #"Spring"
    elif x in [6,7,8]:
        return 2   #"Summer"
    else:
        return 3   #"Autumn"

In [ ]:
train['season']=train['month'].apply(lambda x:get_season(x))
test['season']=test['month'].apply(lambda x:get_season(x))
final_test['season']=final_test['month'].apply(lambda x:get_season(x))

### Quarter start

In [ ]:
def check_if_quater_begin(x):
    """This is used to check if day is begining of quater"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and (month in [1,4,7,9])) else 0

In [ ]:
train['quater_start']=train['date'].apply(lambda x:check_if_quater_begin(x))
test['quater_start']=test['date'].apply(lambda x:check_if_quater_begin(x))
final_test['quater_start']=final_test['date'].apply(lambda x:check_if_quater_begin(x))

### Quarter end

In [ ]:
#Reference https://www.lawinsider.com/dictionary/quarter-end
def check_if_quater_end(x):
    """This is used to check if day is end of quater"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    if (day==31 and month==3) or (day==30 and month==6) or (day==30 and month==9) or (day==31 and month==12):
        return 1
    else:
        return 0

In [ ]:
train['quater_end']=train['date'].apply(lambda x:check_if_quater_end(x))
test['quater_end']=test['date'].apply(lambda x:check_if_quater_end(x))
final_test['quater_end']=final_test['date'].apply(lambda x:check_if_quater_end(x))

### Month start

In [ ]:
def month_start(x):
    """This is used to check if day is begining of month"""
    day=calendar.datetime.date.fromisoformat(x).day
    return 1 if day==1 else 0

In [ ]:
train['month_start']=train['date'].apply(lambda x:month_start(x))
test['month_start']=test['date'].apply(lambda x:month_start(x))
final_test['month_start']=final_test['date'].apply(lambda x:month_start(x))

### Month End

In [ ]:
def month_end(x):
    """This is used to check if day is end of month"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    year=calendar.datetime.date.fromisoformat(x).year
    leap_yr=(year%4==0) # to check if it is a leap year
    val=(day==31 and month==1) or (day==29 if leap_yr else day==28) or (day==31 and month==3) or (day==30 and month==4) or\
        (day==31 and month==5) or (day==30 and month==6) or (day==31 and month==7) or (day==31 and month==8) or\
        (day==30 and month==9) or (day==31 and month==10) or (day==30 and month==11) or (day==31 and month==12)
    return 1 if val else 0

In [ ]:
train['month_end']=train['date'].apply(lambda x:month_end(x))
test['month_end']=test['date'].apply(lambda x:month_end(x))
final_test['month_end']=final_test['date'].apply(lambda x:month_end(x))

### Year start

In [ ]:
def year_start(x):
    """This is used to check if day is begining of year"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and month==1) else 0

In [ ]:
train['year_start']=train['date'].apply(lambda x:year_start(x))
test['year_start']=test['date'].apply(lambda x:year_start(x))
final_test['year_start']=final_test['date'].apply(lambda x:year_start(x))

### Year End

In [ ]:
def year_end(x):
    """This is used to check if day is end of year"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==31 and month==12) else 0

In [ ]:

train['year_end']=train['date'].apply(lambda x:year_end(x))
test['year_end']=test['date'].apply(lambda x:year_end(x))
final_test['year_end']=final_test['date'].apply(lambda x:year_end(x))

### Taking Last 28 days from train for CrossValidation 

In [ ]:
#Cross Validation data will be used for hyperparameter tuning
cv=train[train['date']>='2016-03-28']
train=train[train['date']<'2016-03-28']

In [ ]:
test.to_csv('test.csv',index=False)

In [ ]:
train.to_csv('train.csv',index=False)

In [ ]:
train.shape

(45174237, 28)

In [ ]:
final_test.to_csv('final_test.csv',index=False)

In [ ]:
final_test

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wday,...,snap_WI,sell_price,week_number,season,quater_start,quater_end,month_start,month_end,year_start,year_end
0,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,d_1942,0,2016-05-23,3,...,0,8.38,21,1,0,0,0,0,0,0
1,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,d_1943,0,2016-05-24,4,...,0,8.38,21,1,0,0,0,0,0,0
2,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,d_1944,0,2016-05-25,5,...,0,8.38,21,1,0,0,0,0,0,0
3,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,d_1945,0,2016-05-26,6,...,0,8.38,21,1,0,0,0,0,0,0
4,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,d_1946,0,2016-05-27,7,...,0,8.38,21,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853715,FOODS_3_825_WI_3_evaluation,1434,2,0,9,2,d_1969,0,2016-06-19,2,...,0,3.98,24,2,0,0,0,0,0,0
853716,FOODS_3_826_WI_3_evaluation,1435,2,0,9,2,d_1968,0,2016-06-18,1,...,0,1.28,24,2,0,0,0,0,0,0
853717,FOODS_3_826_WI_3_evaluation,1435,2,0,9,2,d_1969,0,2016-06-19,2,...,0,1.28,24,2,0,0,0,0,0,0
853718,FOODS_3_827_WI_3_evaluation,1436,2,0,9,2,d_1968,0,2016-06-18,1,...,0,1.00,24,2,0,0,0,0,0,0
